In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Charger les données
df = pd.read_csv("dataVisualization/result.csv")

# Supprimer les valeurs manquantes
df.dropna(inplace=True)

# Supprimer les valeurs 'Bee & Bumblebee'
df = df[df['bug type'] != 'Bee & Bumblebee']

# Sélectionner les colonnes de caractéristiques et la cible
train_predictor_columns = df.columns.difference(['bug type', 'species'])
target_labels = df['bug type']
train_feats = df[train_predictor_columns]

# Encodage des étiquettes de la cible
y_encoded, y_labels = pd.factorize(target_labels)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(train_feats, y_encoded, test_size=0.2, random_state=42)

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Définir les meilleurs paramètres trouvés pour MLP
mlp_params = {
    'hidden_layer_sizes': (32, 16, 8),
    'alpha': 0.001,
    'max_iter': 2000
}

# Instancier et entraîner le modèle MLP
mlp = MLPClassifier(solver='adam', random_state=0, tol=1e-9, **mlp_params)
mlp.fit(X_train_scaled, y_train)

# Prédictions MLP
mlp_train_pred = mlp.predict(X_train_scaled)
mlp_test_pred = mlp.predict(X_test_scaled)

# Appliquer SVM avec les meilleurs paramètres trouvés
svm_params = {
    'C': 100,
    'kernel': 'linear',
    'degree': 3,
    'gamma': 'scale',
    'coef0': 0
}
svm = SVC(**svm_params)
svm.fit(X_train_scaled, y_train)

# Prédictions SVM
svm_train_pred = svm.predict(X_train_scaled)
svm_test_pred = svm.predict(X_test_scaled)

# Création de l'ensemble d'entraînement pour le modèle de méta-apprentissage
stacked_train_features = np.column_stack((mlp_train_pred, svm_train_pred))
stacked_test_features = np.column_stack((mlp_test_pred, svm_test_pred))

# Modèle de méta-apprentissage (par exemple, régression logistique)
meta_model = LogisticRegression(random_state=0)
meta_model.fit(stacked_train_features, y_train)

# Prédictions finales
final_test_pred = meta_model.predict(stacked_test_features)

# Obtenir les étiquettes uniques présentes dans les prédictions finales
unique_labels_in_predictions = np.unique(final_test_pred)
target_names = y_labels[unique_labels_in_predictions]

# Évaluation de la performance
classification_rep = classification_report(y_test, final_test_pred, target_names=target_names)
print("Classification Report:")
print(classification_rep)

# Isoler les lignes de result.csv qui ont été utilisées pour tester le modèle
test_df = df.loc[X_test.index]
test_df.drop(train_predictor_columns, axis=1, inplace=True)
test_df.drop(['species'], axis=1, inplace=True)

# Création du DataFrame avec les meilleurs résultats
results_df = pd.DataFrame({
    'bug type': y_labels[y_test],
    'Predicted_Bug_Type': y_labels[final_test_pred],
    'Recognition': pd.Series(y_labels[y_test]) == pd.Series(y_labels[final_test_pred])
})

# Convertir les booléens en chaînes "True" ou "False"
test_df['Stacked_Recognition'] = results_df['Recognition'].map({True: 'True', False: 'False'})
test_df.to_csv("stacked_recognition.csv", index=False)


c:\Users\leona\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


ValueError: Number of classes, 5, does not match size of target_names, 4. Try specifying the labels parameter